
# Advanced Scheduling of Slices

This notebook demonstrates how to schedule slices in advance by specifying a future time range for resource reservation. 
Advanced scheduling allows you to request resources for a specific interval in the future and ensures that the requested resources are 
available for the required duration within that window.

The notebook covers how to:
- Specify a time range for the slice reservation.
- Set a lease duration (in hours) to indicate how long the resources should be available.
- Submit the slice.


## 1: Import the FABlib Library

This section imports the FABlib library to interact with the FABRIC testbed.

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
from fabrictestbed_extensions.fablib.constants import Constants
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

fablib = fablib_manager()

fablib.show_config();


## 2: List the available resources by time

By default, `list_sites` displays the current availability of resources. Additionally, you can check resource availability for a specific time duration.

NOTE: Users do not need to specify an end date and can check resource availability starting from a specific start date.

To query resource availability over a specific time window and focus on particular resources, you can define the relevant fields in the `list_sites` method using the `fields` parameter. In this slice, since we are requesting SmartNICs, we specifically include `nic_connectx_6_available` and `nic_connectx_5_available` in the filter.

In [ ]:
fields=['name','cores_available','ram_available','disk_available','nic_connectx_6_available', 'nic_connectx_5_available']

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

start = (datetime.now(timezone.utc) + timedelta(days=1))
end = start + timedelta(days=1)

print(f"Start Time: {start}")
print(f"End Time: {start}")

In [ ]:
output_table = fablib.list_sites(start=start, end=end, fields=fields)

## Advanced Resource Scheduling

### 3: Set the parameters
Define the name of the slice to be created or loaded.

In [ ]:
# Define parameters (modify these as needed)
slice_name = "AdvancedSchedulingSlice"  # Name for the new slice

node1_name = 'Node1'
node2_name = 'Node2'

network1_name='net1'

model = "NIC_ConnectX_5"

site1 = "LOSA"
site2 = "ATLA"

### 4: Define the Time Duration
Define the duration over which resource availability will be evaluated. This duration represents the continuous period of time for which resources must be available to meet the reservation requirements. For example, in the following code snippet, a 6-hour reservation window is specified, meaning the system will search within the given `start` and `end` time range to identify a contiguous 6-hour block where resources are available for the requested slice.

**Use the `start` time identified in Step 2, based on resource availability, to define the `start` and `end` times here.**

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

start = (datetime.now(timezone.utc) + timedelta(days=1))
end = start + timedelta(days=3)
lease_in_hours = 6

## 5. Create the Experiment Slice

In this example, we will create a slice with nodes configured with `NIC_ConnectX_6` NICs and establish network connections between them.

Dedicated NICs may not always be available at the desired sites, so we are requesting a slice within a specified future time interval for a set duration.

Once a future slice is allocated, it will enter the `AllocatedOK` state. The resources in the slice will then be provisioned and become active at the designated lease start time.

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Network
net1 = slice.add_l2network(name=network1_name, subnet="192.168.1.0/24")

node1 = slice.add_node(name=node1_name, site=site1)
node1_iface1 = node1.add_component(model=model, name='nic1').get_interfaces()[0]

node1_iface1.set_mode('config')
net1.add_interface(node1_iface1)
node1_iface1.set_ip_addr(IPv4Address("192.168.1.1"))

node2 = slice.add_node(name=node2_name, site=site2)
node2_iface1 = node2.add_component(model=model, name='nic1').get_interfaces()[0]

node2_iface1.set_mode('config')
net1.add_interface(node2_iface1)
node2_iface1.set_ip_addr(IPv4Address("192.168.1.2"))


# Add Post Boot Tasks to the Nodes to execute once Lease Start becomes current
node1.add_post_boot_execute('sudo dnf install -y git')
node1.add_post_boot_execute('git clone https://github.com/kthare10/fabric-recipes.git')
node1.add_post_boot_execute('sudo fabric-recipes/node_tools/host_tune.sh')
node1.add_post_boot_execute('fabric-recipes/node_tools/enable_docker.sh')
node1.add_post_boot_execute('docker pull fabrictestbed/slice-vm-rocky8-multitool:0.0.2 ')


node2.add_post_boot_execute('sudo dnf install -y git')
node2.add_post_boot_execute('git clone https://github.com/kthare10/fabric-recipes.git')
node2.add_post_boot_execute('sudo fabric-recipes/node_tools/host_tune.sh')
node2.add_post_boot_execute('fabric-recipes/node_tools/enable_docker.sh')
node2.add_post_boot_execute('docker pull fabrictestbed/slice-vm-rocky8-multitool:0.0.2 ')


node1.add_post_boot_execute("docker run -d --rm "
                            "--network host "
                            "fabrictestbed/slice-vm-rocky8-multitool:0.0.2 "
                            f"iperf3 -s -1 > {node1.get_name()}.log 2>&1");


node2.add_post_boot_execute("docker run --rm "
                            "--network host "
                            "fabrictestbed/slice-vm-rocky8-multitool:0.0.2 "
                            f"iperf3 -c 192.168.1.1 -P 4 -t 30 -i 10 -O 10 > {node2.get_name()}.log 2>&1");

slice.submit(lease_start_time=start, lease_end_time=end, lease_in_hours=lease_in_hours)

## 6. Wait for Slice to become stable (Optional, Blocking)

For slices scheduled in advance, the nodes transition to an active state only once the **Lease Start Time** is reached. Instead of manually waiting for an email notification about the slice status changing from `AllocatedOK` to `StableOK`, users can automate this process.

By using `slice.wait_ssh()`, a script can wait for SSH access to become available—indicating that the nodes are fully booted and ready. At that point, **post-boot tasks**, such as launching an experiment (assuming it has been pre-scripted), can be automatically executed on the nodes.

This allows users to **schedule the slice and experiment in advance**, walk away, and simply return later to collect the results—enabling fully unattended, time-synchronized experimentation.

Note: This is a blocking operation and serves as an example of how it can be executed programmatically when run via a script or a notebook through [Local Jupyter Container](https://learn.fabric-testbed.net/knowledge-base/advanced-jupyter-hub/#why-run-jupyterhub-locally).

In [ ]:
slice = fablib.get_slice(slice_name)

# Determine the wait timeout
timeout = (datetime.strptime(slice.get_lease_start(), Constants.LEASE_TIME_FORMAT) - datetime.now(timezone.utc)).total_seconds() + 1500

print(f"Waiting for timeout: {timeout} seconds")

slice.wait_ssh(timeout=timeout)

slice.post_boot_config()

## 7. Check the experiment results
Once the experiment has completed, you can return at any time to review the results, analyze output files, or inspect logs generated by your post-boot tasks. This enables a fully automated workflow where the experiment runs unattended, and results are available for analysis later.

In [ ]:
stdout, stderr = node1.execute(f"cat {node1.get_name()}.log")
stdout, stderr = node2.execute(f"cat {node2.get_name()}.log")

## 8. Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice.delete()